In [1]:
import re
import os
from tqdm import tqdm
import shutil
from os.path import join
from datetime import datetime

def rm_s(stem:str) :
    "반복문 처음(stem)과 끝(name,extra)에서 실행하라. 언더봐와 스페이스 > strip된 스페이스"
    #시작,중간,끝 모든 공백과 _를 > 하나의 공백으로 > strip하면 끝
    p_s = re.compile("^[_\s)]+|[_\s(]+$|[_\s]{1,}") #(로 끝나거나 )로 시작하는 경우는 제거. 짝 안 맞는 건 주)가 있으니 그냥 두자..
    stem = p_s.sub(" ", stem).strip()
    return stem

def rmNeedless(extra:str) :
    """name과 extra가 확정되면 그때 한 번 해준다. 공백으로 대체가 있으므로 rm_s()를 내포했다.
    일련번호와 기호 제거"""
       
    #완전제거                                                      영어와 숫자가 연속
    p_rmSerialN = re.compile(r"(?<![a-zA-Z])[a-zA-Z](?![a-zA-Z])|\d+[a-zA-Z]+|[a-zA-Z]+\d+|TAA\(회\)|\
        |SCSB|ADMIN.*Conflict|\d(?!건|통|차|채|자|억|천|백|급|번|길|명)") # 모든 숫자를 지운다. 해당글자가 나온다면 그 앞 숫자는 살린다. 
    p_sign = re.compile(r"[^#㈜()\sa-zA-Zㄱ-ㅎ가-힣\d]|\([^\w]*\)") #반쪽 괄호만 있는 거는 어케 지우지?
    
    extra = p_rmSerialN.sub("", extra)
    extra = p_sign.sub(" ", extra)

    extra = rm_s(extra)

    return extra

def re_name(src: str, dst: str) -> list:
    """
    파일명 변경 후 os.rename대신 사용하는 함수(중복확인과 새넘버링)
    폴더를 바꾸는 거 아니라면 파일명 다를때라는 조건문 다음에 호출
    src : dir + file
    dst : dir + new_name
    return : [src dir, src filename, dst filename, dst dir]
    """
    dir = os.path.split(dst)[0]
    f_name = os.path.split(dst)[1]
    stem = os.path.splitext(f_name)[0]
    ext = os.path.splitext(f_name)[1]

    new_name = stem + ext
    i = 1
    while os.path.exists(dir+"/"+new_name):  # 작업디렉토리가 아니므로 풀경로
        new_name = stem + "_"+"("+str(i)+")"+ext
        i += 1

    if not os.path.exists(dir):
        os.makedirs(dir)  # 미리 만들어뒀으니 mkdir해도 됨
    
    shutil.move(src, join(dir, new_name))

    return [os.path.split(src)[0],os.path.split(src)[1], new_name, dir]

def write_log_csv(log:list, path : str) -> None:
    "2차원이 최적이고 그 이상은 셀안에 리스트로"
    if not os.path.exists(path):
        os.makedirs(path)  

    if len(log) > 0 :
    
        import csv
        
        name = str(datetime.today().strftime("%Y%m%d %H%M%S")) + "_" +str(len(log)) + ".csv" 
                    
        with open (join(path, name), "a", newline="", encoding='utf-8-sig') as p :
            for row in log :
                wr = csv.writer(p)
                wr.writerow(row)
    else :
        print(f'log 파일이 비어있습니다. path : {path}')



p = re.compile("ADMIN.*Conflict")
p1 = re.compile("[A-Z]")
path = r"C:\Users\SL\Desktop\test"
path_log_success = path
cnt = 0
ischanged = False
success = []
for root, __dirs__, files in tqdm(os.walk(path)) :
    for f in files :
        stem, ext = os.path.splitext(f)
        if p.search(stem) :
            stem = p.sub("", stem)
            ischanged = True
        if p1.search(ext) :
            ext = ext.lower()
            ischanged = True
        
        if ischanged :
            stem = rmNeedless(stem)
            stem = rm_s(stem)
            new_name = stem + ext
            success.append(re_name(join(root, f), join(root, new_name)))
        
write_log_csv(success, path_log_success)


            
print(cnt)



1it [00:00, ?it/s]

log 파일이 비어있습니다. path : C:\Users\SL\Desktop\test
0


In [2]:
import pickle # 내장모듈
from os.path import join
from tqdm import tqdm
import os
import pandas as pd

In [2]:
### 보고서용 폴더 트리
pathOfDict = r"D:\0.전산\1.진행중과업\파일서버개편\파일\중복조사"
nameOfDict = "정렬된 sc별 파일정보.pkl"

with open(join(pathOfDict, nameOfDict), "rb") as pkl :
    dict = pickle.load(pkl)
total = len(dict)
print(total) 

991093


In [22]:
keys, extraKeys, remains = [], [], []
sep = os.path.sep
for __key__, list in tqdm(dict.items(), total=total) :
    file, temp, local_dir, extra_dir = None, None, None, None
    src = list[0]["fullPath"]
    dir = os.path.split(src)[0]

    file = os.path.split(src)[1] #
    temp = dir.split(sep, maxsplit=3)
    local_dir = temp[2] #
    if len(temp) > 3 :
        extra_dir = temp[3] #
    
    if list[0]["score"] >= 100000 :
        keys.append([file, local_dir, extra_dir])
    elif list[0]["score"] >= 10000 :
        extraKeys.append([file, local_dir, extra_dir])
    else :
        remains.append([file, local_dir, extra_dir])
print(len(keys), len(extraKeys), len(remains))

100%|██████████| 991093/991093 [00:05<00:00, 165958.03it/s]

201317 324387 465389


In [18]:
print(keys[13100])

['20419188_우미화_원인서류.pdf', '서울', 'OK캐피탈\\OK캐피탈2차']


In [23]:
columns = ["file", "local_dir", "extra_dir"]
key_df = pd.DataFrame(keys, columns=columns)
extraKey_df = pd.DataFrame(extraKeys, columns=columns)
remain_df = pd.DataFrame(remains, columns=columns)
print(len(key_df), len(extraKey_df), len(remain_df))

201317 324387 465389


In [24]:
save_path = r'D:\0.전산\1.진행중과업\파일서버개편\폴더트리 보고서'
key_df.to_excel(join(save_path,"key.xlsx"), encoding='utf-8-sig', index=False)
extraKey_df.to_excel(join(save_path,"extraKey.xlsx"), encoding='utf-8-sig', index=False)
remain_df.to_excel(join(save_path,"remain.xlsx"), encoding='utf-8-sig', index=False)

In [2]:
########### 파일 지운 거 다시 복구
import os
import shutil
import time
import pandas as pd
from tqdm import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime
from upload_v2 import dict_refer, write_log_csv, re_name
import pickle

path_log = r'D:\0.전산\1.진행중과업\파일서버개편\log'

In [29]:
df = pd.read_excel(r'D:\0.전산\1.진행중과업\파일서버개편\폴더트리 보고서\extraKey.xlsx', sheet_name=[1])

In [30]:
df = df[1]

In [32]:
data = df.query('local_dir == "광주 공유폴더"' and 'extra_dir == "모음"' and '일 == "대출원장"' and '이 == "케이에스자산관리"')

In [33]:
data

file local_dir extra_dir     일  \
4823            #1_5610152690319_ANY_TYM_001.tif   광주 공유폴더        모음  대출원장   
13775         N2_5604152551114_TYM_PENCO_001.pdf   광주 공유폴더        모음  대출원장   
13776           #1_5708111530315_ANY_TYM_001.tif   광주 공유폴더        모음  대출원장   
13777         M2_5708111530315_TYM_PENCO_001.pdf   광주 공유폴더        모음  대출원장   
13778         N2_3901262530326_TYM_PENCO_001.pdf   광주 공유폴더        모음  대출원장   
...                                          ...       ...       ...   ...   
21150  08627_2002_476_4_08_000000000000000_3.tif   광주 공유폴더        모음  대출원장   
21151  08627_2002_476_4_08_000000000000000_4.tif   광주 공유폴더        모음  대출원장   
21152  08627_2002_476_A_01_000000000000000_1.tif   광주 공유폴더        모음  대출원장   
21153  08627_2002_476_B_09_000000000000000_1.tif   광주 공유폴더        모음  대출원장   
21154  08627_2002_476_B_09_000000000000000_2.tif   광주 공유폴더        모음  대출원장   

              이       삼           사    오    육    칠  
4823   케이에스자산관리  펜타원인서류  6001001184  NaN  NaN  NaN  
13775  케이에스자산관리  펜타원인서류  6001000015  NaN  NaN  NaN  
13776  케이에스자산관리  펜타원인서류  6001000022  NaN  NaN  NaN  
13777  케이에스자산관리  펜타원인서류  6001000022  NaN  NaN  NaN  
13778  케이에스자산관리  펜타원인서류  6001000022  NaN  NaN  NaN  
...         ...     ...         ...  ...  ...  ...  
21150  케이에스자산관리  펜타원인서류  9009000936  NaN  NaN  NaN  
21151  케이에스자산관리  펜타원인서류  9009000936  NaN  NaN  NaN  
21152  케이에스자산관리  펜타원인서류  9009000936  NaN  NaN  NaN  
21153  케이에스자산관리  펜타원인서류  9009000936  NaN  NaN  NaN  
21154  케이에스자산관리  펜타원인서류  9009000936  NaN  NaN  NaN  

[7381 rows x 10 columns]

In [36]:
src_base = r"\\192.168.0.75\스캔파일\구 스캔파일"
dst_base = r"D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류"
log = []
fail = []
for i, v in tqdm(data.iterrows()) :
    src = join(src_base, v.local_dir, v.extra_dir, v.일, v.이, v.삼, str(v.사), v.file)
    new_name = str(v.사) + " " + v.file
    dst = join(dst_base, new_name)
    try : 
        shutil.copy2(src, dst)
        log.append([src, dst])
    except Exception as e :
        fail.append([src, dst, e.__class__, e])

print(f"{len(log)}개 파일 복사.  {len(fail)}개 파일 복사 실패")
write_log_csv(log, dst_base, "성공")
write_log_csv(fail, dst_base, "실패")
    

7381it [20:51,  5.90it/s]

7381개 파일 복사.  0개 파일 복사 실패
log 파일이 비어있습니다. path : D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류


In [41]:
test = pd.read_csv(r"D:\0.전산\1.진행중과업\파일서버개편\log\20221026 105812_4523_광주 모음 대출원장 케이에스 펜타.csv", header=None)
test.columns = ["dir", "file", "일", "이"]
test

dir  \
0     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
1     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
2     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
3     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
4     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
...                                           ...   
4518  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
4519  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
4520  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
4521  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   
4522  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류   

                                                   file  \
0         6001000084 N2_7609172067438_TYM_PENCO_001.pdf   
1              6001000087 #1_4178132771_ANY_TYM_001.tif   
2              6001000087 #2_4178132771_ANY_TYM_001.tif   
3                 6001000087 206007100540669100_김막순.tif   
4                 6001000087 206007100540669100_윤두남.tif   
...                                                 ...   
4518  9009000936 08627_2002_476_4_08_000000000000000...   
4519  9009000936 08627_2002_476_4_08_000000000000000...   
4520  9009000936 08627_2002_476_A_01_000000000000000...   
4521  9009000936 08627_2002_476_B_09_000000000000000...   
4522  9009000936 08627_2002_476_B_09_000000000000000...   

                                                      일  \
0     20414562 6001000084 N2_7609172067438_TYM_PENCO...   
1     20411220 6001000087 #1_4178132771_ANY_TYM_001.tif   
2     20411220 6001000087 #2_4178132771_ANY_TYM_001.tif   
3        20411220 6001000087 206007100540669100_김막순.tif   
4        20411220 6001000087 206007100540669100_윤두남.tif   
...                                                 ...   
4518  20429422 9009000936 08627_2002_476_4_08_000000...   
4519  20429422 9009000936 08627_2002_476_4_08_000000...   
4520  20429422 9009000936 08627_2002_476_A_01_000000...   
4521  20429422 9009000936 08627_2002_476_B_09_000000...   
4522  20429422 9009000936 08627_2002_476_B_09_000000...   

                                                이  
0     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
1     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
2     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
3     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
4     D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
...                                           ...  
4518  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
4519  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
4520  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
4521  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  
4522  D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류  

[4523 rows x 4 columns]

In [46]:
files = list(test.file)

In [49]:
files[0][11:]

'N2_7609172067438_TYM_PENCO_001.pdf'

In [60]:
error = []
for f in files :
    if any(data.file.str.extract(f[11:])) :
        pass
    else :
        error.append(f)
print(len(error))

C:\Users\SL\AppData\Local\Temp\ipykernel_43944\2222372003.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if any(data.file.str.contains(f[11:])) :


720


In [61]:
error

['6001000087 206007100540669100_김막순.tif',
 '6001000087 206007100540669100_윤두남.tif',
 '6001000483 005905060674400100,32.PDF',
 '6001000694 02445_2002_4302_B_01_000000000000000.tif',
 '6001000694 02445_2002_4302_B_29_000000000000000.tif',
 '6001000694 02445_2002_4302_D_02_000000000000000.tif',
 '6001000694 02445_2002_4302_D_03_000000000000000.tif',
 '6001000694 02445_2002_4302_D_06_000000000000000.tif',
 '6001000707 09252_2000_36298_4_02_000000000000000.tif',
 '6001000707 09252_2000_36298_4_08_000000000000000.tif',
 '6001000707 09252_2000_36298_4_08_000000000000000_(7).tif',
 '6001000707 09252_2000_36298_4_08_000000000000000_(8).tif',
 '6001000707 09252_2000_36298_4_08_000000000000000_(9).tif',
 '6001000707 09252_2000_36298_B_01_000000000000000.tif',
 '6001000707 09252_2000_36298_B_02_000000000000000.tif',
 '6001000707 09252_2000_36298_B_99_000000000000000.tif',
 '6001000707 09252_2000_36298_D_03_000000000000000.tif',
 '6001000707 09252_2000_36298_D_05_000000000000000.tif',
 '6001000707 

### 원인서류 옮기기

In [1]:
from tqdm import tqdm
import shutil
import pandas as pd
from os.path import join
from distutils.dir_util import copy_tree
from file_function import write_log_csv_upload
import traceback
import re

dict_refer = pd.read_pickle(r"D:\0.전산\전산조회\dict_refer_20221220.pkl")

In [2]:
df = pd.read_excel(r"C:\Users\SL\Documents\하이웍스 받은파일\추가원인서류 확인필요건.xlsx")
df.head(3)

채무자키        계좌키 Unnamed: 2  추가 원인서류 확인 필요
0  20408800  200945613     개인회생인가            NaN
1  20408866  200945778     개인회생인가            NaN
2  20408854  200945755     개인회생인가            NaN

In [3]:
keylist = df.채무자키.astype(int).astype(dtype=str).drop_duplicates()
total = len(keylist)
total

3482

In [4]:
###############################################
src_base = r"\\192.168.0.75\솔림헬프\1.원인서류"
dst_base = r"D:\220429 추가 원인서류"
###############################################
success, fail = [], []

for key in tqdm(keylist, total=total) :

    try :
        depth2 = dict_refer[key][0]  ### 2(매각사)
        src = join(src_base, depth2, key)
        dst = join(dst_base, depth2, key)
        copy_tree(src, dst)
        success.append([key, src, dst])
    except Exception as e:
        fail.append([key, src, dst, e])
        continue

write_log_csv_upload(success, dst_base, "success")
write_log_csv_upload(fail, dst_base, "fail")

100%|██████████| 3482/3482 [36:12<00:00,  1.60it/s]  


#### 실패건 재확인

In [5]:
df_fail = pd.read_csv(r"C:\Users\SL\Desktop\20221227 180319_1605_fail.csv", header=None)
df_fail.columns = ["key", "src", "dst"]
df_fail.head(3)

key                                         src  \
0  20408433  \\192.168.0.75\솔림헬프\1.원인서류\BNK-01\20408433   
1  20408454  \\192.168.0.75\솔림헬프\1.원인서류\BNK-01\20408454   
2  20408470  \\192.168.0.75\솔림헬프\1.원인서류\BNK-01\20408470   

                                   dst  
0  D:\220429 8번차입 원인서류\BNK-01\20408433  
1  D:\220429 8번차입 원인서류\BNK-01\20408454  
2  D:\220429 8번차입 원인서류\BNK-01\20408470

In [8]:
success, fail = [], []
dst_base = r"C:\Users\SL\Desktop"
for k, v in tqdm(df_fail.iterrows(), total=len(df_fail)) :

    try :
        copy_tree(v.src, v.dst)
        success.append([v.key, v.src, v.dst])
    except Exception as e:
        fail.append([v.key, v.src, v.dst, e])
        continue

write_log_csv_upload(success, dst_base, "success_실패건")
write_log_csv_upload(fail, dst_base, "fail_실패건")

  0%|          | 0/1605 [00:00<?, ?it/s]

cannot copy tree '\\192.168.0.75\솔림헬프\1.원인서류\BNK-01\20408433': not a directory ------------------
<class 'distutils.errors.DistutilsFileError'>
Traceback (most recent call last):
  File "C:\Users\SL\AppData\Local\Temp\ipykernel_21812\909938795.py", line 6, in <cell line: 3>
    copy_tree(v.src, v.dst)
  File "c:\Users\sl\anaconda3\lib\site-packages\setuptools\_distutils\dir_util.py", line 139, in copy_tree
    raise DistutilsFileError("cannot copy tree '%s': not a directory" % src)
distutils.errors.DistutilsFileError: cannot copy tree '\\192.168.0.75\솔림헬프\1.원인서류\BNK-01\20408433': not a directory



#### 파일 복사 결과

In [27]:
path_base = r"D:\220429 8번차입 원인서류"
path_excel = join(path_base, "원인서류list.xlsx")
df_result = pd.read_excel(path_excel)


for k, v in tqdm(df_result.iterrows(), total=len(df_result)) :
    key = str(v.채무자키)
    depth2 = dict_refer[key][0]
    path = join(path_base, depth2, key)
    try : 
        files = [f.name for f in os.scandir(path) if f.is_file()]
        df_result.loc[k, "파일수"] = len(files)
    except FileNotFoundError :
        df_result.loc[k, "파일수"] = 0

df_result.to_excel(path_excel, index=False)

100%|██████████| 14782/14782 [00:02<00:00, 4955.07it/s]


#### 비교

In [20]:
df1 = df.drop_duplicates(subset="채무자키")
len(df1)

10459

In [27]:
df_compare = pd.merge(df1, df_result, on="채무자키", how="inner")
len(df_compare)

10459

In [28]:
df_compare.to_excel(r"C:\Users\SL\Desktop\compare_inner.xlsx", index=False)

#### 관리제외 확인

In [24]:
path_excel = r"D:\220429 8번차입 원인서류\원인서류list.xlsx"
path_base = r"\\192.168.0.75\삭제예정파일\관리제외"
df_result = pd.read_excel(path_excel)


for k, v in tqdm(df_result.iterrows(), total=len(df_result)) :
    key = str(v.채무자키)
    depth2 = dict_refer[key][0]
    path = join(path_base, depth2, key)
    try : 
        for f in os.listdir(path) :
            cnt = 0
            if re.search("원인서류", f) :
                cnt += 1
        df_result.loc[k, "관리제외 파일수"] = cnt
    except FileNotFoundError :
        df_result.loc[k, "관리제외 파일수"] = 0

df_result.to_excel(path_excel, index=False)

100%|██████████| 14782/14782 [02:26<00:00, 100.61it/s]


##### 관리제외 폴더에서 추가하기

In [6]:
# 검색 결과 바탕으로 관리제외인 것만 추가하기
dst_base = r"D:\220429 추가 원인서류"
path_excel = join(dst_base, "원인서류list.xlsx")
df = pd.read_excel(path_excel)

keylist = df[df["관리제외 파일수"]>0].채무자키.astype(int).astype(dtype=str).drop_duplicates() ##########################
total = len(keylist)
total

src_base = r"\\192.168.0.75\삭제예정파일\관리제외"
success, fail = [], []

for key in tqdm(keylist, total=total) :
    try :
        depth2 = dict_refer[key][0]  ### 2(매각사)
        src_dir = join(src_base, depth2, key)

        for f in os.listdir(src_dir) :
            if re.search("원인서류", f) :
                src = join(src_dir, f)
                dst_dir = join(dst_base, depth2, key)
                if not os.path.exists(dst_dir) : os.makedirs(dst_dir)
                
                dst = join(dst_dir, f)
                shutil.copy(src, dst)
                success.append([key, src, dst])
            else : pass
    except Exception as e:
        fail.append([key, src, dst, e])
        continue

write_log_csv_upload(success, dst_base, "관리제외건success")
write_log_csv_upload(fail, dst_base, "관리제외건fail")

100%|██████████| 57/57 [00:36<00:00,  1.58it/s]

log 파일이 비어있습니다. path : D:\220429 추가 원인서류


#### 채무자키 기준 

In [8]:
dst_base = r"D:\220429 8번차입 원인서류"
path_excel = join(dst_base, "원인서류list.xlsx")
df1 = pd.read_excel(path_excel).drop_duplicates(subset="채무자키")

dst_base2 = r"D:\220429 추가 원인서류"
path_excel2 = join(dst_base2, "원인서류list.xlsx")
df2 = pd.read_excel(path_excel2).drop_duplicates(subset="채무자키")

In [20]:
df1 = df1[(df1.파일수>0)|df1["관리제외 파일수"]>0]
df1

순번       구분   매각사구분      채무자키        계좌키   이름            주민번호  \
0          1      무담보  BNK-01  20407815  200941070  송O현  820609-1******   
1          2      무담보  BNK-01  20407818  200941280  최O영  710716-2******   
2          3      무담보  BNK-01  20408432  200940440  정O영  560209-1******   
4          5      무담보  BNK-01  20408444  200940589  정O혜  831008-2******   
5          6      무담보  BNK-01  20408445  200940607  황O하  880819-1******   
...      ...      ...     ...       ...        ...  ...             ...   
14776  14777  개인회생 인가      대산  20408884  200945804  김O정  780616-2******   
14777  14778  개인회생 인가      대산  20408869  200945781  김O애  810404-2******   
14778  14779  개인회생 인가  산은캐-01  20408555  200941512  안O석  741023-1******   
14779  14780  개인회생 인가    한울가람  20408362  200939387  김O두  750801-1******   
14780  14781  개인회생 인가      대산  20408794  200945593  김O후  681205-2******   

       2022.04월 기준  파일수  관리제외 파일수  
0          9531485    1         0  
1         11629822    2         0  
2          5353471    1         0  
4          9760904    1         0  
5          6208288    1         0  
...            ...  ...       ...  
14776       707550    2         0  
14777       104116    0         1  
14778       135244    1         0  
14779        13751    0         1  
14780       357230    2         0  

[9121 rows x 10 columns]

In [21]:
df2 = df2[(df2.파일수>0)|df2["관리제외 파일수"]>0]
df2

채무자키        계좌키 Unnamed: 2  파일수  관리제외 파일수
0     20408800  200945613     개인회생인가    2         0
1     20408866  200945778     개인회생인가    2         0
2     20408854  200945755     개인회생인가    2         0
3     20408064  200922064     개인회생인가    0         1
4     20408955  200945931     개인회생인가    2         0
...        ...        ...        ...  ...       ...
3527  20452934  200976639   신용회복 미확정    2         0
3528  20452935  200976640   신용회복 미확정    1         0
3529  20426682  200945494   신용회복 미확정    2         0
3530  20430452  200950899   신용회복 미확정    2         0
3531  20430460  200950907   신용회복 미확정    2         0

[3427 rows x 5 columns]

In [23]:
df3 = pd.merge(df1, df2, on="채무자키", how="outer")
df3

순번   구분   매각사구분      채무자키        계좌키_x   이름            주민번호  \
0      1.0  무담보  BNK-01  20407815  200941070.0  송O현  820609-1******   
1      2.0  무담보  BNK-01  20407818  200941280.0  최O영  710716-2******   
2      3.0  무담보  BNK-01  20408432  200940440.0  정O영  560209-1******   
3      5.0  무담보  BNK-01  20408444  200940589.0  정O혜  831008-2******   
4      6.0  무담보  BNK-01  20408445  200940607.0  황O하  880819-1******   
...    ...  ...     ...       ...          ...  ...             ...   
12541  NaN  NaN     NaN  20452934          NaN  NaN             NaN   
12542  NaN  NaN     NaN  20452935          NaN  NaN             NaN   
12543  NaN  NaN     NaN  20426682          NaN  NaN             NaN   
12544  NaN  NaN     NaN  20430452          NaN  NaN             NaN   
12545  NaN  NaN     NaN  20430460          NaN  NaN             NaN   

       2022.04월 기준  파일수_x  관리제외 파일수_x        계좌키_y Unnamed: 2  파일수_y  \
0        9531485.0    1.0         0.0          NaN        NaN    NaN   
1       11629822.0    2.0         0.0          NaN        NaN    NaN   
2        5353471.0    1.0         0.0          NaN        NaN    NaN   
3        9760904.0    1.0         0.0          NaN        NaN    NaN   
4        6208288.0    1.0         0.0          NaN        NaN    NaN   
...            ...    ...         ...          ...        ...    ...   
12541          NaN    NaN         NaN  200976639.0   신용회복 미확정    2.0   
12542          NaN    NaN         NaN  200976640.0   신용회복 미확정    1.0   
12543          NaN    NaN         NaN  200945494.0   신용회복 미확정    2.0   
12544          NaN    NaN         NaN  200950899.0   신용회복 미확정    2.0   
12545          NaN    NaN         NaN  200950907.0   신용회복 미확정    2.0   

       관리제외 파일수_y  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
...           ...  
12541         0.0  
12542         0.0  
12543         0.0  
12544         0.0  
12545         0.0  

[12546 rows x 14 columns]